# A linear program for the diet problem

In [ ]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Consider the problem of choosing prepared foods to meet
certain nutritional requirements.  Suppose that precooked
dinners of the following kinds are available for the following prices per
package:

| | | |
| --- | --- | --- |
|BEEF |beef	|\$ 3.19
|CHK |chicken	|2.59
|FISH |fish	|2.29
|HAM |ham	|2.89
|MCH |macaroni & cheese	|1.89
|MTL |meat loaf	|1.99
|SPG |spaghetti	|1.99
|TUR |turkey	|2.49

These dinners provide the following percentages, per
package, of the minimum daily requirements for vitamins A, C, B1 and B2:

|  |A|C|B1|B2|
| --- | --- | --- | --- | --- |
|BEEF|60%|20%|10%|15%|
|CHK|8|0|20|20|
|FISH|8|10|15|10|
|HAM|40|40|35|10|
|MCH|15|35|15|15|	
|MTL|70|30|15|15|
|SPG|25|50|25|15|
|TUR|60|20|15|10|

The problem is to find the cheapest
combination of packages that will meet a week's requirements 
&ndash;
that
is, at least 700% of the daily requirement for each nutrient.

Let us write $X_{BEEF}$ for the
number of packages of beef dinner to be purchased, $X_{CHK}$ for the number
of packages of chicken dinner, and so forth.  Then the
total cost of the diet will be:

$$
\begin{align}
total cost = & \\
	& 3.19 X_{BEEF} + 2.59 X_{CHK} + 2.29 X_{FISH} + 2.89 X_{HAM} + \\
	& 1.89 X_{MCH} + 1.99 X_{MTL} + 1.99 X_{SPG} + 2.49 X_{TUR}
\end{align}
$$

The total percentage of the vitamin A requirement is given by a similar
formula, except that $X_{BEEF}$, $X_{CHK}$, and so forth are multiplied by the
percentage per package instead of the cost per package:

$$
\begin{align}
\text{total percentage of vitamin A daily requirement met} = \\
	60 X_{BEEF} + 8 X_{CHK} + 8 X_{FISH} + 40 X_{HAM} + \\
	15 X_{MCH} + 70 X_{MTL} + 25 X_{SPG} + 60 X_{TUR}
\end{align}
$$

This amount needs to be greater than or equal to
700 percent.  There is a similar formula for each of the other vitamins, and
each of these also needs to be $>=$ 700.

Putting these all together, we have the following linear program:

$$
\begin{align}
\text{Minimize} & \\
& 3.19 X_{BEEF} + 2.59 X_{CHK} + 2.29 X_{FISH} + 2.89 X_{HAM} + \\
& 1.89 X_{MCH} + 1.99 X_{MTL} + 1.99 X_{SPG} + 2.49 X_{TUR} \\
\text{Subject to} & \\
& 60 X_{BEEF} + 8 X_{CHK} + 8 X_{FISH} + 40 X_{HAM} + \\
& 15 X_{MCH} + 70 X_{MTL} + 25 X_{SPG} + 60 X_{TUR} >= 700 \\
& 20 X_{BEEF} + 0 X_{CHK} + 10 X_{FISH} + 40 X_{HAM} + \\
& 35 X_{MCH} + 30 X_{MTL} + 50 X_{SPG} + 20 X_{TUR} >= 700 \\
& 10 X_{BEEF} + 20 X_{CHK} + 15 X_{FISH} + 35 X_{HAM} + \\
& 15 X_{MCH} + 15 X_{MTL} + 25 X_{SPG} + 15 X_{TUR} >= 700 \\
& 15 X_{BEEF} + 20 X_{CHK} + 10 X_{FISH} + 10 X_{HAM} + \\
& 15 X_{MCH} + 15 X_{MTL} + 15 X_{SPG} + 10 X_{TUR} >= 700 \\
& X_{BEEF} >= 0, X_{CHK} >= 0, X_{FISH} >= 0, X_{HAM} >= 0, \\
& X_{MCH} >= 0, X_{MTL} >= 0, X_{SPG} >= 0, X_{TUR} >= 0
\end{align}
$$

At the end we have added the common-sense requirement that no fewer
than zero packages of a food can be purchased.

As we first did with the production LP of Tutorial 1, we can transcribe to a
file, say
diet0.mod ,
an `AMPL` statement of the explicit diet LP:

In [2]:
%%writefile diet0.mod

var Xbeef >= 0; var Xchk >= 0; var Xfish >= 0;
var Xham >= 0;  var Xmch >= 0; var Xmtl >= 0;
var Xspg >= 0;  var Xtur >= 0;

minimize cost:
  3.19*Xbeef + 2.59*Xchk + 2.29*Xfish + 2.89*Xham +
  1.89*Xmch  + 1.99*Xmtl + 1.99*Xspg  + 2.49*Xtur;

subject to A:
  60*Xbeef +  8*Xchk +  8*Xfish + 40*Xham +
  15*Xmch  + 70*Xmtl + 25*Xspg  + 60*Xtur >= 700;

subject to C:
  20*Xbeef +  0*Xchk + 10*Xfish + 40*Xham +
  35*Xmch  + 30*Xmtl + 50*Xspg  + 20*Xtur >= 700;

subject to B1:
  10*Xbeef + 20*Xchk + 15*Xfish + 35*Xham +
  15*Xmch  + 15*Xmtl + 25*Xspg  + 15*Xtur >= 700;

subject to B2:
  15*Xbeef + 20*Xchk + 10*Xfish + 10*Xham +
  15*Xmch  + 15*Xmtl + 15*Xspg  + 10*Xtur >= 700;

Overwriting diet0.mod


Again a few `AMPL` commands then suffice to read the file, send the LP to
a solver, and retrieve the results:

In [3]:
ampl = AMPL()
ampl.read("diet0.mod")
ampl.solve(solver="highs")
ampl.display("Xbeef", "Xchk", "Xfish", "Xham", "Xmch", "Xmtl", "Xspg", "Xtur")

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 88.2
2 simplex iterations
0 barrier iterations
 
Xbeef = 0
Xchk = 0
Xfish = 0
Xham = 0
Xmch = 46.6667
Xmtl = 0
Xspg = 0
Xtur = 0



The optimal solution is found quickly, but it is
hardly what we might have hoped for.  The cost is minimized by a
monotonous diet of 46 2/3 packages of macaroni and cheese!  You can
check that this neatly provides 15% $\times$ 46 2/3 = 700% of the requirement for
vitamins A, B1 and B2, and a lot more vitamin C than necessary; the cost
is only $\$$1.89 $\times$ 46 2/3 = $\$$88.20.
(The tiny negative values for meat loaf and spaghetti can be regarded as zeros,
like the tiny positive values we saw in Section @Tut1@.@adding@.)

You might guess that a better solution would be generated by
requiring the amount of each vitamin to equal 700% exactly.  Such a
equality constraint
requirement can easily be imposed by changing each
$>=$
to
$=$
in the `AMPL`
constraints.  If you go ahead and solve the changed LP, you will find that
the diet does indeed become more varied: approximately 19.5 packages of
chicken, 16.3 of macaroni and cheese, and 4.3 of meat loaf.  But since
equalities are more restrictive than inequalities, the cost goes up to
$\$$89.99.